In [2]:
!pip install requests lxml[html_clean] readability-lxml weasyprint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.1/849.1 kB 32.5 MB/s eta 0:00:00


In [27]:
import requests
from lxml import html
from readability import Document
from weasyprint import HTML

# Define a configuration dictionary
CONFIG = {
    "font_size": 10,
    "heading_emphasizing_factor": 2.5,
    "bg_color": "white",
    "text_color": "black",
    "font_family": "Arial, sans-serif",
    "page_size": "A4",
    "margin": "15mm",
    "line_height": 1.5,
}

def get_print_css(config=CONFIG):
    """
    Generates a customizable CSS string for optimizing web pages for print.
    """
    print_css = f"""
@media print {{
    /* Set background and text properties */
    body {{
        background-color: {config['bg_color']} !important;
        color: {config['text_color']} !important;
        font-size: {config['font_size']}pt;
        font-family: {config['font_family']};
        line-height: {config['line_height']};
        margin: 0;
        padding: 0;
    }}
    
    /* Hide navigation, sidebars, ads, and footers */
    nav, aside, .navigation, .sidebar, .advertisement, .ads, .footer {{
        display: none !important;
    }}
    
    /* Set page size and margins */
    @page {{
        size: {config['page_size']};
        margin: {config['margin']};
    }}
    
    /* Ensure images fit within the print area */
    img {{
        max-width: 100% !important;
        height: auto !important;
        page-break-inside: avoid;
        display: block;
        margin: auto;
    }}
     
    /* Ensure links show their URLs */
    a {{
      text-decoration: none;
    }}
    a[href]:after {{
      content: " (" attr(href) ")";
    }}

    /* Improve heading visibility and structure */
    .main-heading {{
        font-size: {config['font_size'] * config['heading_emphasizing_factor'] * 1.3}pt !important; 
        font-weight: bold !important;
        text-align: left;
        display: block !important;
        margin-top: 20px !important;
        color: {config['text_color']} !important;
    }}
    
    h1 {{
        font-size: {config['font_size'] * config['heading_emphasizing_factor']}pt !important; 
        font-weight: bold !important;
        text-align: left;
        display: block !important;
        margin-top: 20px !important;
        color: {config['text_color']} !important;
    }}
    
    h2 {{
        font-size: {config['font_size'] * config['heading_emphasizing_factor'] * 0.8}pt !important; 
        font-weight: bold !important;
        text-align: left;
        margin-top: 15px !important;
        color: {config['text_color']} !important;
    }}
    
    h3 {{
        font-size: {config['font_size'] * config['heading_emphasizing_factor'] * 0.6}pt !important; 
        font-weight: bold;
        text-align: left;
        margin-top: 10px;
        color: {config['text_color']};
    }}
}}
"""
    return print_css

def fetch_html(url):
    """
    Fetch raw HTML from a given URL with a user-agent header to bypass bot protection.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.text

def extract_main_content(raw_html, url):
    """
    Use readability-lxml to extract the main article content.
    Returns the short title and a cleaned HTML summary.
    """
    doc = Document(raw_html, url=url)
    title = doc.short_title()
    content_html = doc.summary(html_partial=True)
    return title, content_html

def build_full_html(title, content_html, url):
    """
    Wrap the extracted content in a minimal HTML document and include the title and URL in the body.
    """
    full_html = f"""<!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="utf-8">
        <title>{title}</title>
        <style>
            .url-text {{
                color: blue;
            }}
        </style>
    </head>
    <body>
        <h1 class="main-heading">{title}</h1>
        {content_html}
        <hr>
        <p class="url-text">Source URL: {url}</p>
    </body>
    </html>"""
    return full_html

def inject_print_css(html_content, print_css):
    """
    Inject print CSS into the HTML content.
    """
    tree = html.fromstring(html_content)
    head = tree.find("head")
    
    if head is None:
        head = html.Element("head")
        tree.insert(0, head)
    style = html.Element("style")
    style.text = print_css
    head.append(style)
    return html.tostring(tree, pretty_print=True, encoding="unicode")

def generate_pdf(html_content, output_pdf="output.pdf"):
    """
    Convert the given HTML content into a PDF using WeasyPrint.
    """
    HTML(string=html_content).write_pdf(output_pdf)

def main(url, output_pdf="output.pdf", config=CONFIG):
    """
    Main function to generate PDF from a webpage.
    """
    # Generate print-specific CSS using the configuration dictionary
    print_css = get_print_css(config)
    # Fetch the page HTML
    raw_html = fetch_html(url)
    # Extract the main content from the HTML
    title, content_html = extract_main_content(raw_html, url)
    # Build a full HTML document with the extracted content
    full_html = build_full_html(title, content_html, url)
    # Inject the print CSS into the HTML
    final_html = inject_print_css(full_html, print_css)
    # Convert the final HTML to a PDF file
    generate_pdf(final_html, output_pdf)
    print(f"PDF generated successfully: {output_pdf}")

In [29]:
url1 = "https://www.spiceworks.com/tech/artificial-intelligence/articles/what-is-ml/"
url2 = "https://www.vegrecipesofindia.com/paneer-bhurji-scrambled-cottage-cheese-with-spices/#h-about-paneer-bhurji"
url3 = "https://in.indeed.com/hire/c/info/ai-in-business?gad_source=1&gclid=Cj0KCQjwytS-BhCKARIsAMGJyzpWiFtLtPDyDphEvwR_p0vwIKkIW-Gc1waoYCrLdRyjX2M0KoXfL6UaAnptEALw_wcB&aceid=&gclsrc=aw.ds"
url4 = "https://www.geeksforgeeks.org/web-browser/"

if __name__ == "__main__":
    main(url1)

PDF generated successfully: output.pdf
